In [1]:
##Robs summariser of RC8 regoinal Contributor tables
import pandas as pd
import numpy as np
import os

#mainPath = 'P:\projects\RC9_ResultsSets\RegContribTables'
mainPreDevPath = r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth'
localBaselinePath = r'E:\GBRFScenarios\RC10_BASELINE_REGCONTRIBS'
scenarioRegContribsPath = r'E:\GBRFScenarios\Results'
outPath = r'E:\GBRFScenarios\Results\Summaries'
summariesOutFolder = 'SummariesWithProcess'
#baseFolderEnd = 'BASE_RC10'
#changeFolderEnd = 'CHANGE_RC10'
predevFolderEnd = 'PREDEV_RC10'
regContributorFileName = 'RegContributorDataGrid.csv'
#outputCSVEnd = 'RegToOutlet_Alt.csv'
outputCSVEnd = 'RegToOutlet.csv'
baselineScenarioName = 'Baseline'
#changeScenarioName = 'Change'
predevScenarioName = 'PreDev'


pathToContstitsToGrpFile = r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\constituentsToGroups.csv'

annLoadToStream = 'AnnLoadToStreamKG'
annLoadToExport = 'AnnLoadToExportKG'

#regionIDs = ['BM', 'FI', 'MW']
regionIDs = {'BM':['BM_GBRF_MARYRIV_STREAMBANK12p5PERC'], 'FI':['FI_GBRF_FITZRIV_STREAMBANK10PERC','FI_GBRF_FITZRIV_20DTOC',
                'FI_GBRF_MACKRIV_40DTOC'],'MW':['MW_GBRF_DIN_CtoB','MW_GBRF_DIN_DtoB']}

standardScenariosDict = {baselineScenarioName:localBaselinePath, predevScenarioName:mainPreDevPath}
#scenarioToFolderDict = {baselineScenarioName:baseFolderEnd, changeScenarioName:changeFolderEnd, predevScenarioName:predevFolderEnd}


In [2]:
if not os.path.exists(outPath):
    os.makedirs(outPath)
    print("Made dir: " + outPath)

In [3]:
if not os.path.exists(outPath + '\\' + summariesOutFolder):
    os.makedirs(outPath + '\\' + summariesOutFolder)
    print("Made dir: " + outPath + '\\' + summariesOutFolder)

Made dir: E:\GBRFScenarios\Results\Summaries\SummariesWithProcess


In [4]:
def getPreDevPathInfo(regionIDString, scenarioString):
    fileIn = mainPreDevPath + '\\' + regionIDString + '\\Model_Outputs\\' + regionIDString + '_' + predevFolderEnd + '\\' + regContributorFileName
    return fileIn

In [5]:
def getBaselinePathInfo(regionIDString, scenarioString):
    fileIn = localBaselinePath + '\\' + regionIDString + '\\' + regContributorFileName
    return fileIn

In [6]:
def getScenarioPathInfo(regionIDString, scenarioString):
    fileIn = scenarioRegContribsPath + '\\' + regionIDString + '\\' + scenarioString + '_' + regContributorFileName
    return fileIn

In [7]:
def getOutPathInfo(regionIDString, scenarioString):
    fileOut = outPath + '\\' + regionIDString + '_' + scenarioString + '_' + outputCSVEnd
    return fileOut

In [8]:
def produceSummaryFromRegContrib(regionIDString, scenarioName, theFileIn, theFileOut):
    
    #theFileOut = getOutPathInfo(regionIDString, scenarioName)
    
    print("Processing " + theFileIn + " Scenario: " + scenarioName)
    
    rawcontrib = pd.read_csv(theFileIn)
    
    rawcontrib['SCENARIO'] = scenarioName
    rawcontrib[annLoadToStream] = rawcontrib['LoadToStream (kg)'].div(rawcontrib['Num_Days']).mul(365.25)
    rawcontrib[annLoadToExport] = rawcontrib['LoadToRegExport (kg)'].div(rawcontrib['Num_Days']).mul(365.25)
    rawcontrib['AreaHA'] = rawcontrib['AreaM2'].div(10000)
    
    ### Can keep reporting region in the area table, as this is our Non-Nested notebook
    catchFuArea = pd.DataFrame(rawcontrib.groupby(['Rep_Region','ModelElement','FU']).agg({'AreaHA':'first'})).reset_index()
    regFuArea = pd.DataFrame(catchFuArea.groupby(['Rep_Region','FU']).agg({'AreaHA':'sum'})).reset_index()
    
    #regLuseSummary = pd.DataFrame(rawcontrib[rawcontrib['Constituent'].isin(selectedConstituents)].groupby(['Rep_Region','Constituent','FU','Process']).agg({'AnnLoadToStreamKG':'sum','AnnLoadToExportKG':'sum'})).reset_index()
    regLuseSummary = pd.DataFrame(rawcontrib.groupby(['SCENARIO','Rep_Region','Constituent','FU','Process']).agg({annLoadToStream:'sum',annLoadToExport:'sum'})).reset_index()
    
    ### Join/merge with Regional Areas
    regLuseSumPlusArea = pd.merge(regLuseSummary, regFuArea, how='left', left_on=['Rep_Region','FU'], right_on = ['Rep_Region','FU'])
    
    regLuseSumPlusArea.to_csv(theFileOut, index=False)
    
    print("Saved " + theFileOut)
    
    return

    

In [9]:
#Do standard scenarios first

for theReg in regionIDs:
    
    print("Doing: " + theReg + " " + baselineScenarioName)
    baseInFile = getBaselinePathInfo(theReg, baselineScenarioName)
    baseOutFile = getOutPathInfo(theReg, baselineScenarioName)
    produceSummaryFromRegContrib(theReg, baselineScenarioName, baseInFile, baseOutFile)
    
    print("Doing: " + theReg + " " + predevScenarioName)
    predevInFile = getPreDevPathInfo(theReg, predevScenarioName)
    predevOutFile = getOutPathInfo(theReg, predevScenarioName)
    produceSummaryFromRegContrib(theReg, predevScenarioName, predevInFile, predevOutFile)
    
    #Now scenarios
    
    for theScen in regionIDs[theReg]:
        
        print("Doing: " + theReg + " " + theScen)
        scenInFile = getScenarioPathInfo(theReg, theScen)
        scenOutFile = getOutPathInfo(theReg, theScen)
        produceSummaryFromRegContrib(theReg, theScen, scenInFile, scenOutFile)


print("Finished creating individual summary files")

Doing: BM Baseline
Processing E:\GBRFScenarios\RC10_BASELINE_REGCONTRIBS\BM\RegContributorDataGrid.csv Scenario: Baseline
Saved E:\GBRFScenarios\Results\Summaries\BM_Baseline_RegToOutlet.csv
Doing: BM PreDev
Processing \\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\BM\Model_Outputs\BM_PREDEV_RC10\RegContributorDataGrid.csv Scenario: PreDev
Saved E:\GBRFScenarios\Results\Summaries\BM_PreDev_RegToOutlet.csv
Doing: BM BM_GBRF_MARYRIV_STREAMBANK12p5PERC
Processing E:\GBRFScenarios\Results\BM\BM_GBRF_MARYRIV_STREAMBANK12p5PERC_RegContributorDataGrid.csv Scenario: BM_GBRF_MARYRIV_STREAMBANK12p5PERC
Saved E:\GBRFScenarios\Results\Summaries\BM_BM_GBRF_MARYRIV_STREAMBANK12p5PERC_RegToOutlet.csv
Doing: FI Baseline
Processing E:\GBRFScenarios\RC10_BASELINE_REGCONTRIBS\FI\RegContributorDataGrid.csv Scenario: Baseline
Saved E:\GBRFScenarios\Results\Summaries\FI_Baseline_RegToOutlet.csv
Doing: FI PreDev
Processing \\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTru

In [10]:
#Build relationships of constituent names to groups
#tssGrp = 'TSS'
#tnGrp = 'TN'
#tpGrp = 'TP'
#ps2Grp = 'PSII'
#nonps2Grp = 'non-PSII'
##Nope, will read in a CSV to DataFrame instead, thus joing will 'drop' flow from this comparison
#constToGrp = {'Sediment - Fine':tssGrp}

constToGrp = pd.read_csv(pathToContstitsToGrpFile)

baseSupplyStr = 'Base_Supply_KG'
baseExportStr = 'Base_Export_KG'
changeSupplyStr = 'Change_Supply_KG'
changeExportStr = 'Change_Export_KG'
predevSupplyStr = 'PreDev_Supply_KG'
predevExportStr = 'PreDev_Export_KG'
repRegStr = 'Rep_Region'
sourceRegStr = 'Source Region'
constituentStr = 'Constituent'
fuStr = 'FU'
groupStr = 'GROUP'
modelRegStr = 'MODELREG'
scenarioStr = 'SCENARIO'
percRedStr = 'PercReduction'
regTotalStr = 'REGIONAL TOTAL'
processStr = 'Process'

#allExportsAndReductions = pd.DataFrame(columns=[modelRegStr, sourceRegStr, constituentStr, groupStr, predevExportStr, baseExportStr, changeExportStr, percRedStr])
#allExportsAndReductions = pd.DataFrame()


In [15]:
#So now we would be reading in known summary files

def calculateReductions(thisRegID, thisScenario):
    
    theBaseSummaryFile = getOutPathInfo(thisRegID, baselineScenarioName)
    theChangeSummaryFile = getOutPathInfo(thisRegID, thisScenario)
    thePreDevSummaryFile = getOutPathInfo(thisRegID, predevScenarioName)
    
    #print("Reading: " + theBaseSummaryFile)
    baseSummaryDF = pd.read_csv(theBaseSummaryFile)
    #print("Reading: " + theChangeSummaryFile)
    changeSummaryDF = pd.read_csv(theChangeSummaryFile)
    #print("Reading: " + thePreDevSummaryFile)
    predevSummaryDF = pd.read_csv(thePreDevSummaryFile)
    
    #inner drops out those that don't merge, left keep them
    baseTotals = pd.merge(baseSummaryDF.groupby([repRegStr,constituentStr,fuStr,processStr]).agg({annLoadToStream:'sum', annLoadToExport:'sum', 'AreaHA':'first'}).reset_index(), constToGrp, how='inner', left_on=[constituentStr], right_on = ['CONSTITUENT']).reset_index()
    baseTotals = baseTotals.rename(columns={repRegStr:sourceRegStr,'AreaHA':'AREA_HA',annLoadToStream:baseSupplyStr, annLoadToExport:baseExportStr})
    baseTotals = baseTotals[[sourceRegStr, groupStr, constituentStr, fuStr, processStr, 'AREA_HA', baseSupplyStr, baseExportStr]]
    #baseTotals.head(40)
    
    predevTotals = pd.merge(predevSummaryDF.groupby([repRegStr,constituentStr,fuStr,processStr]).agg({annLoadToStream:'sum', annLoadToExport:'sum'}).reset_index(), constToGrp, how='inner', left_on=[constituentStr], right_on = ['CONSTITUENT']).reset_index()
    predevTotals = predevTotals.rename(columns={repRegStr:sourceRegStr,annLoadToStream:predevSupplyStr, annLoadToExport:predevExportStr})
    predevTotals = predevTotals[[sourceRegStr, constituentStr, fuStr, processStr, predevSupplyStr, predevExportStr]]
    
    changeTotals = pd.merge(changeSummaryDF.groupby([repRegStr,constituentStr,fuStr,processStr]).agg({annLoadToStream:'sum', annLoadToExport:'sum'}).reset_index(), constToGrp, how='inner', left_on=[constituentStr], right_on = ['CONSTITUENT'])
    changeTotals = changeTotals.rename(columns={repRegStr:sourceRegStr,annLoadToStream:changeSupplyStr, annLoadToExport:changeExportStr})
    changeTotals = changeTotals[[sourceRegStr, constituentStr, fuStr, processStr, changeSupplyStr, changeExportStr]]
    #changeTotals.head(8)
    
    #, columns=['Rep_Region', 'GROUP', 'Constituent', 'FU', 'AREA_HA', 'Base_Supply_KG', 'Base_Export_KG', 'PreDev_Supply_KG', 'PreDev_Export_KG', 'Change_Supply_KG', 'Change_Export_KG']
    anthroRegByLuse = pd.merge(pd.merge(baseTotals, predevTotals, how='left', left_on=[sourceRegStr,constituentStr,fuStr,processStr],
                                        right_on = [sourceRegStr,constituentStr,fuStr,processStr]).reset_index(), changeTotals, how='left',
                               left_on=[sourceRegStr,constituentStr,fuStr,processStr],
                               right_on = [sourceRegStr,constituentStr,fuStr,processStr])
    
    anthroRegByLuse[scenarioStr] = thisScenario
    #anthroRegByLuse.head(8)
    
    theFileOut = outPath + '\\' + summariesOutFolder + '\\' + thisRegID + '_' + thisScenario + '_AnthroRegByLanduseProcess.csv'
    anthroRegByLuse.to_csv(theFileOut, index=False)
    #print("Saved " + theFileOut)
    
    regExports = anthroRegByLuse.groupby([sourceRegStr, scenarioStr, constituentStr, groupStr]).agg({predevExportStr:'sum', baseExportStr:'sum', changeExportStr:'sum'})
    
    regExports[modelRegStr] = thisRegID
    regExports[percRedStr] = (regExports[baseExportStr] - regExports[changeExportStr]).div(regExports[baseExportStr] - regExports[predevExportStr]).mul(100)
    regExports = regExports.reset_index()
    regExports = regExports[[modelRegStr, sourceRegStr, scenarioStr, constituentStr, groupStr, predevExportStr, baseExportStr, changeExportStr, percRedStr]]
    #regExports.head(15)
    
    modelRegExports = regExports.groupby([modelRegStr, scenarioStr, constituentStr, groupStr]).agg({predevExportStr:'sum', baseExportStr:'sum', changeExportStr:'sum'})
    modelRegExports[sourceRegStr] = regTotalStr
    modelRegExports[percRedStr] = (modelRegExports[baseExportStr] - modelRegExports[changeExportStr]).div(modelRegExports[baseExportStr] - modelRegExports[predevExportStr]).mul(100)
    modelRegExports = modelRegExports.reset_index()
    modelRegExports = modelRegExports[[modelRegStr, sourceRegStr, scenarioStr, constituentStr, groupStr, predevExportStr, baseExportStr, changeExportStr, percRedStr]]
    ##modelRegExports.head(15)
    regExports = regExports.append(modelRegExports, ignore_index=True)
    
    return regExports


In [17]:
allExportsAndReductions = pd.DataFrame(columns=[modelRegStr, sourceRegStr, scenarioStr, constituentStr, groupStr, predevExportStr, baseExportStr, changeExportStr, percRedStr])

for theReg in regionIDs:
    for scen in regionIDs[theReg]:
        
        theseExports = calculateReductions(theReg, scen)
        allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)


# theseExports = calculateReductions('Burdekin', 'BU')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

# theseExports = calculateReductions('Burnett Mary', 'BM')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

# theseExports = calculateReductions('Cape York', 'CY')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

# theseExports = calculateReductions('Fitzroy', 'FI')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

# theseExports = calculateReductions('Mackay Whitsunday', 'MW')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

# theseExports = calculateReductions('Wet Tropics', 'WT')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

reqVals = ['TSS', 'TN', 'TP']

allExportsAndReductions = allExportsAndReductions.loc[allExportsAndReductions[groupStr].isin(reqVals)]

theFileNameOut = outPath + '\\' + summariesOutFolder + '\\GBRF_AllExportsAndReductions_FromProcess.csv'
allExportsAndReductions.to_csv(theFileNameOut, index=False)
allExportsAndReductions.head(20)
#print(allExportsAndReductions)

,MODELREG,Source Region,SCENARIO,Constituent,GROUP,PreDev_Export_KG,Base_Export_KG,Change_Export_KG,PercReduction
11,BM,Baffle,BM_GBRF_MARYRIV_STREAMBANK12p5PERC,N_DIN,TN,2.589656e+04,6.375811e+04,6.375811e+04,0.0
12,BM,Baffle,BM_GBRF_MARYRIV_STREAMBANK12p5PERC,N_DON,TN,1.131271e+05,2.136307e+05,2.136307e+05,0.0
13,BM,Baffle,BM_GBRF_MARYRIV_STREAMBANK12p5PERC,N_Particulate,TN,5.921790e+04,1.549179e+05,1.549179e+05,0.0
14,BM,Baffle,BM_GBRF_MARYRIV_STREAMBANK12p5PERC,P_DOP,TP,4.689431e+03,8.435126e+03,8.435126e+03,0.0
15,BM,Baffle,BM_GBRF_MARYRIV_STREAMBANK12p5PERC,P_FRP,TP,7.415385e+03,1.392016e+04,1.392016e+04,0.0
16,BM,Baffle,BM_GBRF_MARYRIV_STREAMBANK12p5PERC,P_Particulate,TP,3.700617e+04,1.004474e+05,1.004474e+05,0.0
20,BM,Baffle,BM_GBRF_MARYRIV_STREAMBANK12p5PERC,Sediment - Fine,TSS,2.109436e+07,6.471098e+07,6.471098e+07,0.0
33,BM,Burnett,BM_GBRF_MARYRIV_STREAMBANK12p5PERC,N_DIN,TN,3.759815e+04,2.447733e+05,2.447733e+05,0.0
34,BM,Burnett,BM_GBRF_MARYRIV_STREAMBANK12p5PERC,N_DON,TN,1.642445e+05,4.570025e+05,4.570025e+05,0.0
35,BM,Burnett,BM_GBRF_MARYRIV_STREAMBANK12p5PERC,N_Particulate,TN,4.923534e+05,9.591656e+05,9.591656e+05,0.0
